In [199]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix


In [200]:
def zscore_per_gene_samplesxgenes(X):
    return (X - X.mean(axis=0)) / X.std(axis=0, ddof=0)
    

In [201]:
# Expression: rows=samples, cols=Ensembl IDs
expr = pd.read_csv("../../output/nr_gx.csv", index_col=0)

# Labels: columns Samples, Group
lab = pd.read_csv("../../output/clusters.csv")
y = lab.set_index("Samples")["Group"].reindex(expr.index)

# keep only labeled samples
keep = y.dropna().index
expr = expr.loc[keep]        
y = y.loc[keep]

# 6 overlap genes

overlap_genes = [
    "ENSG00000174945","ENSG00000124882","ENSG00000254772",
    "ENSG00000115604","ENSG00000197467","ENSG00000139269",
]

# Genes are in COLUMNS now
genes = [g for g in overlap_genes if g in expr.columns]   # <-- FIX: expr.columns not expr.index
missing = sorted(set(overlap_genes) - set(genes))
print("Genes present:", genes)
print("Genes missing:", missing)

# X_train is just the selected gene columns (no transpose)
X_train = expr.loc[:, genes]   # <-- FIX: subset columns
X_train_z = zscore_per_gene_samplesxgenes(X_train)
y_train = y.map({1: 0, 2: 1}).astype(int)
y_train = y_train.astype(int)        # keep only if already 0/1

print("X_train:", X_train_z.shape, "y_train:", y_train.shape)
print("Class counts:\n", y_train.value_counts())

Genes present: ['ENSG00000174945', 'ENSG00000124882', 'ENSG00000254772', 'ENSG00000115604', 'ENSG00000197467', 'ENSG00000139269']
Genes missing: []
X_train: (39, 6) y_train: (39,)
Class counts:
 Group
0    23
1    16
Name: count, dtype: int64


In [202]:

model = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel="linear", C=5))
])
model.fit(X_train_z, y_train)


Pipeline(steps=[('scaler', StandardScaler()),
                ('svm', SVC(C=5, kernel='linear'))])

In [203]:
# Expression: rows=Ensembl IDs, cols=samples
Xext_raw = pd.read_csv("/home/hocine/myProjs/students/Taye/geo/output/GSE17538/L_2.csv", index_col=0)

# Labels: columns Samples, Group
y_ext = pd.read_csv("/home/hocine/myProjs/students/Taye/geo/output/GSE17538/clusters_2.csv", index_col=0)["Group"]
common = Xext_raw.columns.intersection(y_ext.index)
Xext_raw = Xext_raw[common]
y_ext = y_ext.loc[common]
genes_final = [g for g in overlap_genes if g in Xext_raw.index]
X_test = Xext_raw.loc[genes_final].T
X_test_z  = zscore_per_gene_samplesxgenes(X_test)
print("X_test:", X_test_z.shape, "y_ext:", y_ext.shape)
print("Genes used in external:", genes_final)

X_test: (45, 6) y_ext: (45,)
Genes used in external: ['ENSG00000174945', 'ENSG00000124882', 'ENSG00000254772', 'ENSG00000115604', 'ENSG00000197467', 'ENSG00000139269']


In [204]:
y_test  = y_ext.map({1: 0, 2: 1}).astype(int)
y_test = y_test.astype(int)

In [205]:
scores = model.decision_function(X_test_z)   # continuous scores for ROC AUC
auc = roc_auc_score(y_test, scores)

y_pred = (scores >= 0).astype(int)         # SVM default threshold
acc = accuracy_score(y_test, y_pred)

f1  = f1_score(y_test, y_pred)
cm  = confusion_matrix(y_test, y_pred)

print("External ROC AUC:", auc)
print("External Accuracy:", acc)
print("External F1:", f1)
print("Confusion matrix:\n", cm)

External ROC AUC: 1.0
External Accuracy: 0.9555555555555556
External F1: 0.9333333333333333
Confusion matrix:
 [[29  2]
 [ 0 14]]


In [206]:
import numpy as np

rng = np.random.default_rng(0)
B = 2000
aucs = []
y_np = np.array(y_test)
s_np = np.array(scores)

for _ in range(B):
    idx = rng.integers(0, len(y_np), len(y_np))
    # skip resamples with 1 class (AUC undefined)
    if len(np.unique(y_np[idx])) < 2:
        continue
    aucs.append(roc_auc_score(y_np[idx], s_np[idx]))

lo, hi = np.percentile(aucs, [2.5, 97.5])
print("AUC 95% CI:", (lo, hi))


AUC 95% CI: (0.9999999999999999, 1.0)


In [207]:
# Silhouette

import numpy as np
from sklearn.metrics import silhouette_score, silhouette_samples


# ensure labels are 0/1
# y_sil = y_test.map({1:0, 2:1}).astype(int)
y_sil = np.array(y_test)

# Silhouette requires at least 2 clusters and no cluster of size 1
unique, counts = np.unique(y_sil, return_counts=True)
print("Cluster sizes:", dict(zip(unique, counts)))

# Overall silhouette (mean)
sil_mean = silhouette_score(X_test_z.values, y_sil, metric="euclidean")
print("Mean silhouette width:", sil_mean)

# Per-sample silhouettes (useful to report distribution / per-class mean)
sil_per_sample = silhouette_samples(X_test_z.values, y_sil, metric="euclidean")

# Per-class mean silhouette
for lab in unique:
    m = sil_per_sample[y_sil == lab].mean()
    print(f"Mean silhouette for class {lab}: {m}")

Cluster sizes: {0: 31, 1: 14}
Mean silhouette width: 0.4708564374539533
Mean silhouette for class 0: 0.4876598593172141
Mean silhouette for class 1: 0.4336488604710181
